# Regression Week 5: Feature Selection and LASSO (Interpretation)

In this notebook, you will use LASSO to select features, building on a pre-implemented solver for LASSO (using Turi Create, though you can use other solvers). You will:
* Run LASSO with different L1 penalties.
* Choose best L1 penalty using a validation set.
* Choose best L1 penalty using a validation set, with additional constraint on the size of subset.

In the second notebook, you will implement your own LASSO solver, using coordinate descent. 

# Fire up Turi Create

In [1]:
import turicreate

# Load in house sales data

Dataset is from house sales in King County, the region where the city of Seattle, WA is located.

In [2]:
sales = turicreate.SFrame('../home_data.sframe/')

# Create new features

As in Week 2, we consider features that are some transformations of inputs.

In [3]:
from math import log, sqrt
sales['sqft_living_sqrt'] = sales['sqft_living'].apply(sqrt)
sales['sqft_lot_sqrt'] = sales['sqft_lot'].apply(sqrt)
sales['bedrooms_square'] = sales['bedrooms']*sales['bedrooms']

# In the dataset, 'floors' was defined with type string, 
# so we'll convert them to float, before creating a new feature.
sales['floors'] = sales['floors'].astype(float) 
sales['floors_square'] = sales['floors']*sales['floors']

* Squaring bedrooms will increase the separation between not many bedrooms (e.g. 1) and lots of bedrooms (e.g. 4) since 1^2 = 1 but 4^2 = 16. Consequently this variable will mostly affect houses with many bedrooms.
* On the other hand, taking square root of sqft_living will decrease the separation between big house and small house. The owner may not be exactly twice as happy for getting a house that is twice as big.

# Learn regression weights with L1 penalty

Let us fit a model with all the features available, plus the features we just created above.

In [4]:
all_features = ['bedrooms', 'bedrooms_square',
                'bathrooms',
                'sqft_living', 'sqft_living_sqrt',
                'sqft_lot', 'sqft_lot_sqrt',
                'floors', 'floors_square',
                'waterfront', 'view', 'condition', 'grade',
                'sqft_above',
                'sqft_basement',
                'yr_built', 'yr_renovated']

Applying L1 penalty requires adding an extra parameter (`l1_penalty`) to the linear regression call in Turi Create. (Other tools may have separate implementations of LASSO.)  Note that it's important to set `l2_penalty=0` to ensure we don't introduce an additional L2 penalty.

In [5]:
model_all = turicreate.linear_regression.create(sales, target='price', features=all_features,
                                                validation_set=None, 
                                                l2_penalty=0., l1_penalty=1e10)

Linear regression:

--------------------------------------------------------

Number of examples          : 21613

Number of features          : 17

Number of unpacked features : 17

...

| 5         | 6        | 0.000002  | 1.431937     | 6869667.895833     | 385998.458623                   |

| 10        | 11       | 0.000002  | 1.558610     | 6842123.232651     | 364204.576180                   |

+-----------+----------+-----------+--------------+--------------------+---------------------------------+

Completed (Iteration limit reached).

This model may not be optimal. To improve it, consider increasing `max_iterations`.

Find what features had non-zero weight.

In [6]:
model_all

Class                          : LinearRegression

Schema
------
Number of coefficients         : 18
Number of examples             : 21613
Number of feature columns      : 17
Number of unpacked features    : 17

Hyperparameters
---------------
L1 penalty                     : 10000000000.0
L2 penalty                     : 0.0

Training Summary
----------------
Solver                         : fista
Solver iterations              : 10
Solver status                  : Completed (Iteration limit reached).
Training time (sec)            : 1.567

Settings
--------
Residual sum of squares        : 2842629034369063.0
Training RMSE                  : 364204.5762

Highest Positive Coefficients
-----------------------------
(intercept)                    : 274873.056
bathrooms                      : 8468.5311
grade                          : 842.068
sqft_living_sqrt               : 350.0606
sqft_living                    : 24.4207

Lowest Negative Coefficients
----------------------------
No Ne

In [9]:
model_all.coefficients['value'].nnz()

6

Note that a majority of the weights have been set to zero. So by setting an L1 penalty that's large enough, we are performing a subset selection. 

***QUIZ QUESTION***:
According to this list of weights, which of the features have been chosen? 

# Selecting an L1 penalty

To find a good L1 penalty, we will explore multiple values using a validation set. Let us do three way split into train, validation, and test sets:
* Split our sales data into 2 sets: training and test
* Further split our training data into two sets: train, validation

Be *very* careful that you use seed = 1 to ensure you get the same answer!

In [10]:
(training_and_validation, testing) = sales.random_split(.9,seed=1) # initial train/test split
(training, validation) = training_and_validation.random_split(0.5, seed=1) # split training into train and validate

Next, we write a loop that does the following:
* For `l1_penalty` in [10^1, 10^1.5, 10^2, 10^2.5, ..., 10^7] (to get this in Python, type `np.logspace(1, 7, num=13)`.)
    * Fit a regression model with a given `l1_penalty` on TRAIN data. Specify `l1_penalty=l1_penalty` and `l2_penalty=0.` in the parameter list.
    * Compute the RSS on VALIDATION data (here you will want to use `.predict()`) for that `l1_penalty`
* Report which `l1_penalty` produced the lowest RSS on validation data.

When you call `linear_regression.create()` make sure you set `validation_set = None`.

Note: you can turn off the print out of `linear_regression.create()` with `verbose = False`

In [11]:
import numpy as np

In [12]:
l1_penalty_list = np.logspace(1, 7, num=13)

In [13]:
#no degree
def get_RSS(model,test_set,output):
    predicted_values = model.predict(test_set)
    # then compute the residuals (since we are squaring it doesn't matter which order you subtract)
    true_output = test_set[output]
    residual = true_output - predicted_values
    # square the residuals and add them up
    RSS = (residual * residual).sum()
    return(RSS)

In [14]:
#data_set = training
#poly_data = polynomial_sframe(data_set['sqft_living'], 15)
#my_features = poly_data.column_names() # get the name of the features
#poly_data['price'] = data_set['price'] # add price to the data since it's the target
#l2_penalty_list = np.logspace(1, 7, num=13)
model = {}
RSS = {}
for l1_p in l1_penalty_list:
    model[l1_p] = turicreate.linear_regression.create(training, target='price', features=all_features,
                                                validation_set=None, 
                                                l2_penalty=0., l1_penalty=l1_p, verbose = False)
    RSS[l1_p] = get_RSS(model[l1_p], validation, 'price')
l1_penalty_best = min(RSS,key=RSS.get)
print("l1 penalty:", l1_penalty_best,", RSS:" ,'%e'%RSS[l1_penalty_best])

l1 penalty: 10.0 , RSS: 6.257663e+14


In [15]:
RSS

{10.0: 625766285142460.5,
 31.622776601683793: 625766285362394.5,
 100.0: 625766286057885.1,
 316.22776601683796: 625766288257224.4,
 1000.0: 625766295212186.1,
 3162.2776601683795: 625766317206080.8,
 10000.0: 625766386760658.0,
 31622.776601683792: 625766606749278.5,
 100000.0: 625767302791634.9,
 316227.7660168379: 625769507643885.8,
 1000000.0: 625776517727024.5,
 3162277.6601683795: 625799062845466.6,
 10000000.0: 625883719085425.0}

*** QUIZ QUESTION. *** What was the best value for the `l1_penalty`?

***QUIZ QUESTION***
Also, using this value of L1 penalty, how many nonzero weights do you have?

In [56]:
len(all_features)

17

In [57]:
all_features

['bedrooms',
 'bedrooms_square',
 'bathrooms',
 'sqft_living',
 'sqft_living_sqrt',
 'sqft_lot',
 'sqft_lot_sqrt',
 'floors',
 'floors_square',
 'waterfront',
 'view',
 'condition',
 'grade',
 'sqft_above',
 'sqft_basement',
 'yr_built',
 'yr_renovated']

In [60]:
model[l1_penalty_best].coefficients.print_rows(20)

+------------------+-------+----------------------+--------+
|       name       | index |        value         | stderr |
+------------------+-------+----------------------+--------+
|   (intercept)    |  None |   18993.4272127706   |  None  |
|     bedrooms     |  None |  7936.9676790313015  |  None  |
| bedrooms_square  |  None |  936.9933681932994   |  None  |
|    bathrooms     |  None |  25409.588934120668  |  None  |
|   sqft_living    |  None |  39.11513637970764   |  None  |
| sqft_living_sqrt |  None |  1124.650212807717   |  None  |
|     sqft_lot     |  None | 0.003483618222989654 |  None  |
|  sqft_lot_sqrt   |  None |  148.25839101140826  |  None  |
|      floors      |  None |  21204.335466950117  |  None  |
|  floors_square   |  None |  12915.524336072433  |  None  |
|    waterfront    |  None |   601905.594545272   |  None  |
|       view       |  None |  93312.85731187189   |  None  |
|    condition     |  None |  6609.035712447216   |  None  |
|      grade       |  No

# Limit the number of nonzero weights

What if we absolutely wanted to limit ourselves to, say, 7 features? This may be important if we want to derive "a rule of thumb" --- an interpretable model that has only a few features in them.

In this section, you are going to implement a simple, two phase procedure to achive this goal:
1. Explore a large range of `l1_penalty` values to find a narrow region of `l1_penalty` values where models are likely to have the desired number of non-zero weights.
2. Further explore the narrow region you found to find a good value for `l1_penalty` that achieves the desired sparsity.  Here, we will again use a validation set to choose the best value for `l1_penalty`.

In [17]:
max_nonzeros = 7

## Exploring the larger range of values to find a narrow range with the desired sparsity

Let's define a wide range of possible `l1_penalty_values`:

In [18]:
l1_penalty_values = np.logspace(8, 10, num=20)

Now, implement a loop that search through this space of possible `l1_penalty` values:

* For `l1_penalty` in `np.logspace(8, 10, num=20)`:
    * Fit a regression model with a given `l1_penalty` on TRAIN data. Specify `l1_penalty=l1_penalty` and `l2_penalty=0.` in the parameter list. When you call `linear_regression.create()` make sure you set `validation_set = None`
    * Extract the weights of the model and count the number of nonzeros. Save the number of nonzeros to a list.
        * *Hint: `model.coefficients['value']` gives you an SArray with the parameters you learned.  If you call the method `.nnz()` on it, you will find the number of non-zero parameters!* 

In [19]:
model = {}
num_of_nz = {}
for l1_p in l1_penalty_values:
    model[l1_p] = turicreate.linear_regression.create(training, target='price', features=all_features,
                                                validation_set=None, 
                                                l2_penalty=0., l1_penalty=l1_p, verbose = False)
    num_of_nz[l1_p] = model[l1_p].coefficients['value'].nnz()

In [20]:
num_of_nz

{100000000.0: 18,
 127427498.57031322: 18,
 162377673.91887242: 18,
 206913808.111479: 18,
 263665089.87303555: 17,
 335981828.6283788: 17,
 428133239.8719396: 17,
 545559478.1168514: 17,
 695192796.1775591: 17,
 885866790.4100832: 16,
 1128837891.6846883: 15,
 1438449888.2876658: 15,
 1832980710.8324375: 13,
 2335721469.0901213: 12,
 2976351441.6313133: 10,
 3792690190.7322536: 6,
 4832930238.571753: 5,
 6158482110.6602545: 3,
 7847599703.514623: 1,
 10000000000.0: 1}

Out of this large range, we want to find the two ends of our desired narrow range of `l1_penalty`.  At one end, we will have `l1_penalty` values that have too few non-zeros, and at the other end, we will have an `l1_penalty` that has too many non-zeros.  

More formally, find:
* The largest `l1_penalty` that has more non-zeros than `max_nonzeros` (if we pick a penalty smaller than this value, we will definitely have too many non-zero weights)
    * Store this value in the variable `l1_penalty_min` (we will use it later)
* The smallest `l1_penalty` that has fewer non-zeros than `max_nonzeros` (if we pick a penalty larger than this value, we will definitely have too few non-zero weights)
    * Store this value in the variable `l1_penalty_max` (we will use it later)


*Hint: there are many ways to do this, e.g.:*
* Programmatically within the loop above
* Creating a list with the number of non-zeros for each value of `l1_penalty` and inspecting it to find the appropriate boundaries.

In [21]:
for l1_p in l1_penalty_values:
    if num_of_nz[l1_p] >= max_nonzeros:
        l1_penalty_min = l1_p
    else:
        l1_penalty_max = l1_p
        break

***QUIZ QUESTION.*** What values did you find for `l1_penalty_min` and `l1_penalty_max`, respectively? 

In [22]:
print('l1_penalty_min =',l1_penalty_min, ' l1_penalty_max = ',l1_penalty_max)

l1_penalty_min = 2976351441.6313133  l1_penalty_max =  3792690190.7322536


## Exploring the narrow range of values to find the solution with the right number of non-zeros that has lowest RSS on the validation set 

We will now explore the narrow region of `l1_penalty` values we found:

In [23]:
l1_penalty_list = np.linspace(l1_penalty_min,l1_penalty_max,20)

In [24]:
l1_penalty_list

array([2.97635144e+09, 3.01931664e+09, 3.06228184e+09, 3.10524703e+09,
       3.14821223e+09, 3.19117743e+09, 3.23414263e+09, 3.27710782e+09,
       3.32007302e+09, 3.36303822e+09, 3.40600341e+09, 3.44896861e+09,
       3.49193381e+09, 3.53489901e+09, 3.57786420e+09, 3.62082940e+09,
       3.66379460e+09, 3.70675980e+09, 3.74972499e+09, 3.79269019e+09])

* For `l1_penalty` in `np.linspace(l1_penalty_min,l1_penalty_max,20)`:
    * Fit a regression model with a given `l1_penalty` on TRAIN data. Specify `l1_penalty=l1_penalty` and `l2_penalty=0.` in the parameter list. When you call `linear_regression.create()` make sure you set `validation_set = None`
    * Measure the RSS of the learned model on the VALIDATION set

Find the model that the lowest RSS on the VALIDATION set and has sparsity *equal* to `max_nonzeros`.

In [25]:
model_7nz = {}
RSS_7nz = {}
for l1_p in l1_penalty_list:
    model_7nz[l1_p] = turicreate.linear_regression.create(training, target='price', features=all_features,
                                                validation_set=None, 
                                                l2_penalty=0., l1_penalty=l1_p, verbose = False)
    RSS_7nz[l1_p] = get_RSS(model_7nz[l1_p], validation, 'price')
l1_penalty_best = min(RSS_7nz,key=RSS_7nz.get)
print("l1 penalty:", l1_penalty_best,", RSS:" ,'%e'%RSS_7nz[l1_penalty_best])

l1 penalty: 2976351441.6313133 , RSS: 9.669257e+14


In [27]:
for l1_p in l1_penalty_list:
    print("l1 penalty: ", l1_p ,'has ',model_7nz[l1_p].coefficients['value'].nnz(), ' non-zero coefficients with RSS: ',RSS_7nz[l1_p] )

l1 penalty:  2976351441.6313133 has  10  non-zero coefficients with RSS:  966925692362085.2
l1 penalty:  3019316638.9524155 has  10  non-zero coefficients with RSS:  974019450084554.6
l1 penalty:  3062281836.2735176 has  10  non-zero coefficients with RSS:  981188367942453.8
l1 penalty:  3105247033.5946198 has  10  non-zero coefficients with RSS:  989328342459473.5
l1 penalty:  3148212230.915722 has  10  non-zero coefficients with RSS:  998783211265890.1
l1 penalty:  3191177428.236824 has  10  non-zero coefficients with RSS:  1008477167020094.4
l1 penalty:  3234142625.557926 has  10  non-zero coefficients with RSS:  1018298780553819.4
l1 penalty:  3277107822.8790283 has  10  non-zero coefficients with RSS:  1028247992205977.0
l1 penalty:  3320073020.2001305 has  8  non-zero coefficients with RSS:  1034616909232827.4
l1 penalty:  3363038217.5212326 has  8  non-zero coefficients with RSS:  1038554735941040.2
l1 penalty:  3406003414.8423347 has  8  non-zero coefficients with RSS:  1043237

***QUIZ QUESTIONS***
1. What value of `l1_penalty` in our narrow range has the lowest RSS on the VALIDATION set and has sparsity *equal* to `max_nonzeros`?
2. What features in this model have non-zero coefficients?

In [91]:
model_7nz[l1_penalty_best].coefficients.print_rows(20)

+------------------+-------+--------------------+--------+
|       name       | index |       value        | stderr |
+------------------+-------+--------------------+--------+
|   (intercept)    |  None | 196100.93780562267 |  None  |
|     bedrooms     |  None | 2181.574321070672  |  None  |
| bedrooms_square  |  None |        0.0         |  None  |
|    bathrooms     |  None | 17962.696661157832 |  None  |
|   sqft_living    |  None | 34.14246565122559  |  None  |
| sqft_living_sqrt |  None |  789.319789078234  |  None  |
|     sqft_lot     |  None |        0.0         |  None  |
|  sqft_lot_sqrt   |  None |        0.0         |  None  |
|      floors      |  None | 3665.9308175965243 |  None  |
|  floors_square   |  None |        0.0         |  None  |
|    waterfront    |  None |        0.0         |  None  |
|       view       |  None | 11333.841030833135 |  None  |
|    condition     |  None |        0.0         |  None  |
|      grade       |  None | 3578.9004004381122 |  None 

In [93]:
model_7nz[l1_penalty_best].coefficients['value'].nnz()

10

In [29]:
model_7nz[l1_penalty_best].coefficients[model_7nz[l1_penalty_best].coefficients['value'] != 0].head(20)

name,index,value,stderr
(intercept),None,196100.93780562267,None
bedrooms,None,2181.574321070672,None
bathrooms,None,17962.696661157832,None
sqft_living,None,34.14246565122559,None
sqft_living_sqrt,None,789.319789078234,None
floors,None,3665.9308175965243,None
view,None,11333.841030833135,None
grade,None,3578.9004004381122,None
sqft_above,None,32.74320137175839,None
sqft_basement,None,12.795381135944734,None
